In [2]:
#目標一：如何利用bs parser xml檔

from urllib.request import urlopen
from bs4 import BeautifulSoup


xml = open('path',encoding='utf-8')
bs = BeautifulSoup(xml,'xml')

articleNumbers = []

articles = bs.find_all('article')

for article in articles:
    articleNumbers.append(article['no'])

articleIndex = articleNumbers.index('100022')
print(articles[articleIndex].genre)
print(articles[articleIndex].style)
print(articles[articleIndex].find('class'))

#取得所有author下sex, nationality內容
articles[articleIndex].author.sex
articles[articleIndex].author.nationality

#tag名字
articles[articleIndex].author.name 
#content表取得所有author下內容
print(articles[articleIndex].author.content[1]) 


for sen in articles[articleIndex].find_all('sentence'):
    print(sen)


FileNotFoundError: [Errno 2] No such file or directory: 'path'

In [17]:
#目標二：建立model
import requests

class Content:
    def __init__(self, url, title, body): #建構子
        self.url = url
        self.title = title
        self.body = body
        
def getPage(url):
    req = requests.get(url)
    return BeautifulSoup(req.text,'html.parser')

def scrapeNYtime(url):
    bs = getPage(url)
    title = bs.find('h1').text
    lines = bs.find_all('p',{'class':'css-158dogj evys1bk0'})
    body = '\n'.join([line.text for line in lines])
    return Content(url, title, body)

def scrapeBooking(url):
    bs = getPage(url)
    title = bs.find('h1').text
    lines = bs.find('div',{'itemprop':'articleBody'}).find_all('p')
    body = [line.text for line in lines]
    return Content(url, title, body)


url = 'http://www.brookings.edu/blog/future-development/2018/01/26/delivering-inclusive-urban-access-3-uncomfortable-truths'
content = scrapeBooking(url)
print('url:%s' %content.url)
print('title:%s' %content.title)
print('body:%s\n' %content.body)

url = 'https://www.nytimes.com/2018/01/25/opinion/sunday/silicon-valley-immortality.html'
content = scrapeNYtime(url)
print('url:',content.url)
print('title:',content.title)
print('body:',content.body)

url:http://www.brookings.edu/blog/future-development/2018/01/26/delivering-inclusive-urban-access-3-uncomfortable-truths
title:Delivering inclusive urban access: 3 uncomfortable truths
body:['The past few decades have been filled with a deep optimism about the role of cities and suburbs across the world. These engines of economic growth host a majority of world population, are major drivers of economic innovation, and have created pathways to opportunities for untold amounts of people.\t\n\n\n\n\n\n\nJeffrey Gutman\nNonresident Senior Fellow - Global Economy and Development\n\n\n\n\n\n\n\nAdie Tomer\nFellow - Metropolitan Policy Program\n\n Twitter\nAdieTomer\n\n\n\n\n\n', 'But all is not well within our so-called Urban Century. Rapid urbanization, rising gentrification, concentrated poverty, and shortages of basic infrastructure have combined to create spatial inequity in cities and suburbs across the globe. The challenges of housing, moving, and employing so many people have led to l

url: https://www.nytimes.com/2018/01/25/opinion/sunday/silicon-valley-immortality.html
title: The Men Who Want to Live Forever
body: Would you like to live forever? Some billionaires, already invincible in every other way, have decided that they also deserve not to die. Today several biotech companies, fueled by Silicon Valley fortunes, are devoted to “life extension” — or as some put it, to solving “the problem of death.”
It’s a cause championed by the tech billionaire Peter Thiel, the TED Talk darling Aubrey de Gray, Google’s billion-dollar Calico longevity lab and investment by Amazon’s Jeff Bezos. The National Academy of Medicine, an independent group, recently dedicated funding to “end aging forever.”
As the longevity entrepreneur Arram Sabeti told The New Yorker: “The proposition that we can live forever is obvious. It doesn’t violate the laws of physics, so we can achieve it.” Of all the slightly creepy aspects to this trend, the strangest is the least noticed: The people public

In [32]:
import requests

class Content:
    def __init__(self, url, title, body): #建構子
        self.url = url
        self.title = title
        self.body = body
    
    def print(self):
        print('URL:%s' %self.url)
        print('TITLE:%s' %self.title)
        print('BODY:\n%s' %self.body)
        print('-'*4,'next','-'*40)
class Website:
    def __init__(self, name, url, titleTag, bodyTag):
        self.name = name
        self.url = url
        self.titleTag = titleTag
        self.bodyTag = bodyTag

In [33]:
import requests
from bs4 import BeautifulSoup

class Crawler:
    def getPage(self,url):
        try:
            req = requests.get(url)
        except requests.exceptions.RequestException:
            return None
        
        return BeautifulSoup(req.text,'html.parser')
    
    #用select方法取得所要的tag
    def safeGet(self, pageObj, selector):
        selectedElems = pageObj.select(selector)
        if selectedElems is not None and len(selectedElems)>0:
            return '\n'.join([elem.text for elem in selectedElems])
        
    def parse(self, site, url):
        bs = self.getPage(url)
        if bs is not None:
            title = self.safeGet(bs,site.titleTag)
            body = self.safeGet(bs, site.bodyTag)
            
            if title != '' and body !='':
                content = Content(url, title, body)
                content.print()              

In [40]:
crawler = Crawler()

siteData = [
    ['O\'Reilly Media', 'http://oreilly.com', 'h1', 'span>div>p'],
    ['Reuters', 'http://reuters.com', 'h1', 'div.StandardArticleBody_body>p'],
    ['Brookings', 'http://www.brookings.edu', 'h1', 'div.post-body>p'],
    ['New York Times', 'http://nytimes.com', 'h1', 'div.StoryBodyCompanionColumn div p']
]

websites = []
for att in siteData:
    websites.append(Website(att[0],att[1],att[2],att[3]))

crawler.parse(websites[0],'http://shop.oreilly.com/product/0636920028154.do') 
crawler.parse(websites[1],'http://www.reuters.com/article/us-usa-epa-pruitt-idUSKBN19W2D0') 
crawler.parse(websites[2],'https://www.brookings.edu/blog/techtank/2016/03/01/idea-to-retire-old-methods-of-policy-education/') 
crawler.parse(websites[3],'https://www.nytimes.com/2018/01/28/business/energy-environment/oil-boom.html') 
    

URL:http://shop.oreilly.com/product/0636920028154.do
TITLE:Learning Python, 5th Edition
BODY:
Get a comprehensive, in-depth introduction to the core Python language with this hands-on book. Based on author Mark Lutz’s popular training course, this updated fifth edition will help you quickly write efficient, high-quality code with Python. It’s an ideal way to begin, whether you’re new to programming or a professional developer versed in other languages.
Complete with quizzes, exercises, and helpful illustrations,  this easy-to-follow, self-paced tutorial gets you started with both Python 2.7 and 3.3— the latest releases in the 3.X  and 2.X lines—plus all other releases in common use today. You’ll also learn some advanced language features that recently have become more common in Python code.
---- next ----------------------------------------
URL:http://www.reuters.com/article/us-usa-epa-pruitt-idUSKBN19W2D0
TITLE:EPA chief wants scientists to debate climate on TV
BODY:
WASHINGTON (Reute

URL:https://www.nytimes.com/2018/01/28/business/energy-environment/oil-boom.html
TITLE:Oil Boom Gives the U.S. a New Edge in Energy and Diplomacy
BODY:
HOUSTON — A substantial rise in oil prices in recent months has led to a resurgence in American oil production, enabling the country to challenge the dominance of Saudi Arabia and dampen price pressures at the pump.
The success has come in the face of efforts by Saudi Arabia and its oil allies to undercut the shale drilling spree in the United States. Those strategies backfired and ultimately ended up benefiting the oil industry.
Overcoming three years of slumping prices proved the resiliency of the shale boom. Energy companies and their financial backers were able to weather market turmoil — and the maneuvers of the global oil cartel — by adjusting exploration and extraction techniques.
After a painful shakeout in the industry that included scores of bankruptcies and a significant loss of jobs, a steadier shale-drilling industry is ari

In [61]:
#利用topic對某網頁的內容做蒐尋
class Content:    
    def __init__(self, topic, url, title, body):
        self.topic = topic
        self.url = url 
        self.title = title
        self.body = body
        
    def print(self):
        print('New article found for topic:%s' %self.topic)
        print('URL: %s' %self.url)
        print('TITLE:%s' %self.title)
        print('BODY:%s' %self.body)
        
class Website:
    
    def __init__(self, name, url, searchUrl, resultListening,
                 resultUrl, absoluteUrl, titleTag, bodyTag):
        
        self.name = name
        self.url = url
        self.searchUrl = searchUrl
        self.resultListening = resultListening
        self.resultUrl = resultUrl
        self.absoluteUrl = absoluteUrl
        self.titleTag = titleTag
        self.bodyTag = bodyTag

In [ ]:
import requests
from bs4 import BeautifulSoup

class Crawler:
    #回傳bs4
    def getPage(self, url):
        try:
            req = requests.get(url)
        except requests.exceptions.RequestException:
            return None
        return BeautifulSoup(req.text,'html.parser')
    
    #搜尋所要tag
    def safeGet(self, pageObj, selector):
        childObj = pageObj.select(selector)
        if childObj is not None and len(childObj)>0:
            return childObj[0].text
        return ''
    
    #Searches a given website for a given topic and records all pages found
    def search(self, topic, site):
        bs = self.getPage(site.searchUrl + topic)
        
        #找第一層
        searchResults = bs.select(site.resultListening)
        
        #找第二層
        for result in searchResults:
            url = result.select(site.resultUrl)[0].attrs['href']
            
            if (site.absoluteUrl):
                bs = self.getPage(url)
            else:
                bs = self.getPage(site.url+url)
            
            if bs is None:
                print('Something was wrong with that page or URL. Skipping!')
                return
            
            title = self.safeGet(bs, site.titleTag)
            body = self.safeGet(bs, site.bodyTag)
            
            if title != '' and body != '':
                content = Content(topic, url, title, body)
                content.print()
            

In [64]:
crawler = Crawler()

#name, url, searchUrl, resultListening,
#resultUrl, absoluteUrl, titleTag, bodyTag

siteData = [
    ['O\'Reilly Media', 'http://oreilly.com',
    'https://ssearch.oreilly.com/?q=', 'article.product-result', 
    'p.title a', True, 'h1', 'section#product-description'],
    ['Reuters', 'http://reuters.com',
    'http://www.reuters.com/search/news?blob=',
    'div.search-result-content', 'h3.search-result-title a',
    False, 'h1', 'div.StandardARticleBody_body_1gnLA'],
    ['Brookings', 'http://www.brookings.edu',
    'https://www.brookings.edu/search/?s=',
    'div.list-content article', 'h4.title a', True, 'h1', 'div.post-body']
]

sites=[]

for site in siteData:
    sites.append(Website(site[0], site[1], site[2],
                         site[3], site[4], site[5],
                         site[6], site[7]))
    
topics = ['python','data science']

                 
for topic in topics:
    print('Getting Info About:',topic)
    for targetSite in sites:
        crawler.search(topic, targetSite)


Getting Info About: python
New article found for topic:python
URL: The Hutchins Center Explains: Budgeting for aging America
TITLE:


For decades, we have been hearing that the baby-boom generation was like a pig moving through a python–bigger than the generations before and after. 
That’s true. But that’s also a very misleading metaphor for understanding the demographic forces that are driving up federal spending: They aren’t temporary. The generation born between 1946 and 1964 is the beginning of a demographic transition that will persist for decades after the baby boomers die, the consequence of lengthening lifespans and declining fertility. Putting the federal budget on a sustainable course requires long-lasting fixes, not short-lived tweaks.  
First, a few demographic facts.
As the chart below illustrates, there was a surge in births in the U.S. at the end of World War II, a subsequent decline, and then an uptick as baby boomers began having children.




Although the population h

New article found for topic:python
URL: The Silicon Valley Wage Premium
TITLE:


Software application developers earn large salaries in the United States, $96,260 a year on average. But in metropolitan San Jose they earn $131,270, the highest in the country. There are many partial explanations for this—local cost of living, differences in education levels, experience, and industry—but none of them quite account for it. It turns out that developers living in San Jose have acquired the specific skills most valued by employers.
 
As the map below shows, there is a huge amount of variation in earnings for software application developers across regional labor markets. In large metropolitan areas like New York, they earn $105,000, but in Louisville, they earn just $72,000.
 

Average Salary of Software Application Developers by Metropolitan Area, 2013

 


 
Similar patterns could be shown for other occupations, of course; for even within the same job title, people vary by education and expe

New article found for topic:python
URL: Modeling with Data: Tools and Techniques for Scientific Computing
TITLE:

PREFACE


BODY:https://www.brookings.edu/articles/modeling-with-data-tools-and-techniques-for-scientific-computing/
New article found for topic:python
URL: Forum: Debating Bush’s Wars
TITLE:

In the 

Winter 2007–08 issue 
of Survival, Philip Gordon argued that America’s strategy against terror is failing ‘because the Bush administration chose to wage the wrong war’. Survival invited former Bush speechwriter and Deputy Assistant to the President Peter Wehner and Kishore Mahbubani, Dean and Professor at the Lee Kuan Yew School of Public Policy in Singapore, to reflect on Gordon’s arguments. Their 
comments are available in the above PDF and Philip Gordon’s response is below.

BODY:https://www.brookings.edu/articles/forum-debating-bushs-wars/
New article found for topic:python
URL: Think Bigger on North Korea
TITLE:
While the world is fixated on Iraq and the Middle East, Nort

New article found for topic:data science
URL: What all policy analysts need to know about data science
TITLE:







Alex Engler
Rubenstein Fellow - Governance Studies

 Twitter
@AlexCEngler





Conversations around data science typically contain a lot of buzzwords and broad generalizations that make it difficult to understand its pertinence to governance and policy. Even when well-articulated, the private sector applications of data science can sound quite alien to public servants. This is understandable, as the problems that Netflix and Google strive to solve are very different than those government agencies, think tanks, and nonprofit service providers are focused on. This does not mean, however, that there is no public sector value in the modern field of data science. With qualifications, data science offers a powerful framework to expand our evidence-based understanding of policy choices, as well as directly improve service delivery.
To better understand its importance to public 

New article found for topic:data science
URL: Bridging the gender data gap
TITLE:
More men than women are killed in car crashes each year, partly because men drive more and engage in riskier driving behavior. On the other hand, women are 17% more likely to be killed and 47% more likely to be injured in crashes than men are. Women are at increased risk simply because they are women: cars are primarily designed, built, and tested by male engineers using male data, so they are built with men in mind. Scaled-down versions of male crash test dummies, meant to represent women, were not used until 2003—and are primarily tested in the passenger seat. In car design, development, and testing, male bodies are the standard and female bodies the outlier. This creates a gender data gap with very real impacts on the lives of Americans.





J



Jeanette Gaudry Haynie
Founder and Executive Director - Athena Leadership Project Lieutenant Colonel - U.S. Marine Corps Reserve




The gender data gap appe

...

New article found for topic:data science
URL: The opportunities and challenges of data analytics in health care
TITLE:
Data analytics tools have the potential to transform health care in many different ways. In the near future, routine doctor’s visits may be replaced by regularly monitoring one’s health status and remote consultations. The inpatient setting will be improved by more sophisticated quality metrics drawn from an ecosystem of interconnected digital health tools. The care patients receive may be decided in consultation with decision support software that is informed not only by expert judgments but also by algorithms that draw on information from patients around the world, some of whom will differ from the “typical” patient. Support may be customized for an individual’s personal genetic information, and doctors and nurses will be skilled interpreters of advanced ways to diagnose, track, and treat illnesses. In a number of different ways, policymakers are likely to have new t

New article found for topic:data science
URL: Using big data to link poor farmers to finance
TITLE:
Two billion adults in the world are excluded from credit. The situation is especially bad for small farmers in rural areas who are unable to access loans to invest in their farms, trapped in a vicious circle of low productivity, low yields, and poor income. The Initiative for Smallholder Finance estimates that smallholders globally access just $50 billion of the $200 billion of lending that they require to grow their operations and improve their lives.	






Roy Parizat
Fund Manager, BioCarbon Fund Initiative for Sustainable Forest Landscapes - World Bank





H



Heinz-Wilhelm Strubenhoff
Agribusiness Program Manager, World Bank Group





The global growth of microfinance banks has created new opportunities for financial inclusion, with outstanding lending of $100 billion to around 200 million clients. Yet the majority of lending from microfinance institutions has been to urban popul

New article found for topic:data science
URL: Drawing from improvement science to bridge education research and practice
TITLE:
A three-legged stool can be tough to balance if the legs are uneven. We’ve been thinking a lot about how to do just that with the launch of the  Millions Learning Real-time Scaling Labs starting in Brazil, Jordan, Tanzania, Côte d’Ivoire, and the U.S. city of Philadelphia. How do we balance and give equal weight to our three primary objectives: learn from, document, and support education interventions in the process of scaling? We think that drawing from principles behind adaptive and iterative learning methodologies, such as improvement science, can help.	






Jenny Perlman Robinson
Senior Fellow - Global Economy and Development, Center for Universal Education

 Twitter
@JennyPerlman






Our starting point with the scaling labs is to learn more about how effective education interventions expand and deepen their impact—what we refer to as scale—especially 

New article found for topic:data science
URL: Idea to retire: Viewing data as an output
TITLE:
It is rare to talk to federal administrators and not hear something about “data-driven decision making.” Administrators dedicate significant time and information technology resources to decisions about their agency’s viability and to defending their standing in appropriations discussions. However, the questions asked of the data by senior officials for these purposes are dramatically different than the questions asked by a front line case manager for a social service organization.  Unfortunately, it is the former that has the loudest voice in procurement and system design conversations.  The prevailing top-down approach to data collection and use also puts little actionable information in the hands of the people responsible for gathering data. As a result, the incentive to collect data is diminished while opportunities for informed program improvements and abilities to measure outcomes agains

New article found for topic:data science
URL: A new alphabet for Europe: Algorithms, big data, and the computer chip
TITLE:
If the biggest disrupter of the last few decades was Deng Xiaoping—the father of modern China—the big disrupter of the next few decades may well be John McCarthy. McCarthy, an American professor of Computer Science, is believed by many to be the father of artificial intelligence. Interestingly, the two have an epiphany in common. In 1979, Deng, a lifelong communist, visited the United States and came back a believer in market capitalism. In 1968, after a two-day visit to Czechoslovakia, McCarthy, who was raised as a communist by his immigrant parents, became a free-market Republican.
The ideas of computer scientists and mathematicians like McCarthy are radically transforming the way we communicate, and the way we make, buy, and sell goods and services. The changes will likely be so great that societies will have to reorganize government policy—rethinking how to re

In [82]:
#追蹤任何符合某種URL模式的內外部連結

class Website:
    
    def __init__(self, name, url, targetPattern, absoluteUrl, titleTag, bodyTag):
        self.name = name
        self.url = url
        self.targetPattern = targetPattern
        self.absoluteUrl = absoluteUrl
        self.titleTag = titleTag
        self.bodyTag = bodyTag
        
class Content:
    def __init__(self, url, title, body):
        self.url = url
        self.title = title
        self.body = body

    def print(self):
        print('URL:%s' %(self.url))
        print('TITLE:%s' %(self.title))
        print('BODY:%s' %(self.body))

In [110]:
import re
from bs4 import BeautifulSoup
import requests 


class Crawler:
    
    def __init__(self, site):
        self.site = site
        self.visited=[]
        
    def getPage(self, url):
        try:
            req = requests.get(url)
        except requests.exceptions.RequestException:
            return None
        return BeautifulSoup(req.text,'html.parser')
    
    def safeGet(self, pageObj, selector):
        selectElems = pageObj.select(selector)

        if selectElems is not None and len(selectElems)>0:
            return '\n'.join([elem.text for elem in selectElems])
        return ''
    
    def parse(self, url):
        bs = getPage(url)
        if bs is not None:
            
            title = self.safeGet(bs, self.site.titleTag)
            body = self.safeGet(bs, self.site.bodyTag)
            if title != '' and body != '':
                content = Content(url, title, body)
                content.print()
    
    def crawl(self):
        
        bs = self.getPage(self.site.url)
        targetPages = bs.find_all('a',href = re.compile(self.site.targetPattern))
        for targetPage in targetPages:
            targetPage = targetPage['href']
            
            if targetPage not in self.visited:
                self.visited.append(targetPage)
                if not self.site.absoluteUrl:
                    targetPage = '%s%s' %(self.site.url, targetPage)
                self.parse(targetPage)
                


In [111]:
reters = Website('Reuters', 'https://www.reuters.com',
                 '^(/article/)', False,
                 'h1', 'div.StandardArticleBody_body p')

crawler = Crawler(reters)
crawler.crawl()

URL:https://www.reuters.com/article/china-business-record-financing-0629-idCNKBS24011J
TITLE:（更正）焦点：2020上半年中国企业售股筹资金额占全球一半 创纪录新高
BODY:（更正：因英文原文有误，内文第11段中信里昂证券(CLSA)李航头衔应为”全球股票资本市场主管”，非”大中华区股票资本市场主管”） 
路透香港6月29日 - 中国企业今年至今所筹措的股权资本占到全球的半数，创下纪录新高，凸显出当地经济提早走出新型冠状病毒疫情，同时也显示中美关系恶化导致中国企业回归本地市场的程度。 
路孚特数据显示，中国当地企业在1-6月发行了总值321亿美元的股票，其中包括数以十亿美元计的香港第二上市，相当于全球总额的49.8%。美国企业发行股票总额为158亿美元。 
“在多国政府推出大规模挹注流动性措施以援助受疫情打击经济的情形下，我对于中国今年筹资规模并不感到意外，这个趋势可能持续，”Davis Polk资本市场合伙人Li He表示，他称中国企业正运用经济提早解封的优势。 
中国自12月起受到新型冠状病毒疫情冲击，且在1月底成为首个实施防疫封锁措施的国家，限制个人和企业的活动。当地各个市场在4月开始恢复正常运作。 
中国筹资市场既得益于推出已有一年的科创板，亦受惠于反响不错的香港市场首次公开发行(IPO)和第二上市—包括本月的线上零售商京东(JD.O) 9618.HK和游戏开发商网易(NTES.O) 9999.HK，这两笔第二上市交易的筹资规模分别为39亿美元和31亿。 
“对于中国企业而言，香港和美国市场都在回归正常，”摩根大通在中国的全球投资银行业务主管Houston Huang说。”市场活动(交易的数量)远远超过今年年初所有人的预期。” 
贸易等问题导致中美地缘政治紧张局势升温，料将促使更多在美上市的中资企业选择离自己家门口近的市场进行第二上市，这些市场里不存在敌视中国的气氛。 
两位直接知情人士说，考虑在香港进行第二上市的公司包括百胜中国(YUMC.N)和中通快递(ZTO.N)。 
由于不在上班时间，百胜中国和中通快递均未对置评请求作出回覆。 
一些公司在香港进行第二上市，也增加了投资人的兴趣。香港市场向来以传统的金融和地产股为主，而非侧重成长的科技股。 
中信里昂证券(C

URL:https://www.reuters.com/article/global-economy-covid-outlook-0626-idCNKBS24003J
TITLE:综述：新冠疫情反复 全球经济复苏前景难言乐观
BODY:路透班加罗尔6月26日 - 路透调查显示，绝大多数分析师认为，过去一个月全球经济复苏前景恶化，或充其量就是维持不变，持续衰退预期将较上月预估更严重。 
全球股市周四跌至逾一周低点，受许多国家新冠病例激增影响，市场滋生对经济复苏的疑虑。 
这其中就包括美国。美国5月似乎已控制住了疫情，导致几个州解除限制举措。 
疫情二次爆发风险，以及国际货币基金组织(IMF)最新警告称今年全球经济可能萎缩近5%，再加上贸易紧张情势升温，令人担忧金融市场预期的V型复苏可能面临风险。 
路透本月对逾250位分析师调查显示，几个主要发达国家持续的衰退程度将超过上月时的预估，到明年年底时失业率料将远超疫情前的水准。 
但被问及过去一个月全球经济复苏前景有何变化时，近90名分析师中有80%(71名)表示，前景恶化或充其量保持不变。 
其余受访者称前景改善，但是更广泛的调查对今年全球GDP的预估较一个月前下降，而且基本与最坏情况相同。 
“我们认为未来有三种风险。第一种是疫情卷土重来。这可能导致重启进程逆转。第二种风险是货币和/或财政政策疲劳，”美国银行全球经济研究主管Ethan Harris表示。 
“各国政府一直在积极应对这一冲击。但是，货币政策现在已经所剩无几。主要发达经济体可能出台更多财政刺激，但出台速度可能更慢且规模可能更小。第三种风险是第二轮冲击可能比我们想象的更加严重。” 
今年全球经济料将萎缩3.7%，明年可能增长5.4%。前次调查预估分别为萎缩3.2%和增长5.4%。 
尽管今年的预期中值没有IMF预计的萎缩4.9%那么悲观，但最新预期中的最差情境下全球经济料将萎缩6.0%。 
“即使近期出现一些积极迹象，但初期的降幅远远大于2008年后的降幅。从现在到年底可能新增数千万个就业岗位，但就业市场加速复苏并非V形复苏，”巴克莱宏观研究部门主管Ajay Rajadhyaksha称。 
“我们预计要到2021年末全球GDP才会超过疫情前的水平，对发达经济体而言这个时间还会更晚。而且还可能出现更多不利因素，尤其是如果出现第二波疫情的话。” 
与此同时，预计

URL:https://www.reuters.com/article/japan-automakers-may-production-0629-idCNKBS2400VI?il=0
TITLE:日本汽车厂商5月全球销量下滑38% 受疫情封锁措施拖累
BODY:路透东京6月29日 - 日本汽车厂商5月全球销量下降38%，为连续第三个月大幅下滑，新冠疫情封锁管控措施导致多数汽车厂和经销商继续停业。 
路透根据各大公司销售数据计算，上月日本七大汽车厂商共计销售147万辆汽车，较上年同期的238万辆下滑，但降幅小于4月时的暴跌50%。 
这些汽车生产商5月的全球产量锐减62%至918,974辆，4月产量为下降55%。 
许多国家已放宽封锁措施，但行业专家预期，将需要五年时间才能让需求恢复到2019年水准。 
所有地区的汽车销量均呈现下降，北美和欧洲受创最重。中国是唯一亮点，丰田(7203.T)、日产(7201.T)、马自达(7261.T)均报告在中国销量上升。(完) 
编译 郑茵 审校 张涛 
URL:https://www.reuters.com/article/china-securities-stocks-covid-0629-idCNKBS2400T7?il=0
TITLE:中国股市收盘下跌受新一波疫情疑虑打击 券商股领跌
BODY:路透上海6月29日 - 中国股市周一收盘下跌，因国内外新增新冠病毒感染病例都急速上升，引发各界担心中国萌芽中的经济复苏。 
沪深300指数.CSI300收跌0.71%，至4,109.72点，沪综指.SSEC收跌0.61%报2,961.52点。创业板指数.CNT收跌0.417%，受获利了结影响。 
今日是端午节长周末假期过后首日交易。对于全球疫情恶化、死于新冠病毒人数突破50万大关，以及美股周五下跌超过2%等题材，中国投资人做出反应。 
中航证券的分析师在报告中写道，由于疫情反弹改变了市场对于全球经济复苏的乐观情绪，市场将经历新一波震荡。 
虽然中国5月工业企业利润六个月来首次增长，”疫情下的市场需求仍然相对疲弱，利润复苏的可持续性需要进一步观察，”国家统计局工业司高级统计师朱虹表示。 
中国央行周日表示，随着企业重启经济出现好转迹象，但增长仍然面临疫情带来的挑战。 
券商股跌幅居前，此前中国媒体财新获悉，中国证监会计划向商业银行发放券

URL:https://www.reuters.com/article/yuan-midpoint-0629-mon-idCNKBS24002D?il=0
TITLE:路透测算的人民币中间价料开在7.0887元附近 逆周期影响96点
BODY:路透北京6月29日 - 路透根据中国外汇交易中心(CFETS)公布的人民币汇率形成机制测算，人民币兑美元中间价周一预计会开在7.0887元附近，较上日贬值约332点。 
上日人民币兑美元即期CNY=CFXS收报7.0744元，而中间价则为7.0555元。根据定价机制，今日中间价变动中，有189点下跌反映市场供求影响，另有239点下跌反映的是隔夜一篮子汇率基本保持不变所需要的点数；还有96点上涨反映的是“逆周期因子”影响。 
路透测算的上日人民币兑美元中间价为7.0564元，和外汇交易中心公布的实际值偏差9点。上日路透根据CFETS公布的中间价测算，人民币汇率CFETS指数、参考SDR（特别提款权）篮子指数和参考BIS(国际清算银行)篮子指数分别为91.94、90.66和95.73。 
2016年2月以来中国初步形成”收盘汇率+一篮子货币汇率变化”的人民币对美元汇率中间价形成机制，2017年2月将中间价对一篮子货币的参考时段由24小时调整为15小时，5月底外汇市场自律机制汇率工作组在中间价报价模型中引入”逆周期因子”。 
在2018年初中国央行建议各报价行”可以调整”报价模型中的”逆周期系数”，使其恢复中性；6月起人民币快速下跌，市场贬值预期再起，8月24日央行宣布重启逆周期因子。(完)  
发稿 韩笑；审校 张喜良 
URL:https://www.reuters.com/article/csrc-banks-brokerage-response-0629-mon-idCNKBS24001T?il=0
TITLE:中国证监会回应银行将获券商牌照报导 称暂无更多信息向市场通报
BODY:路透上海6月29日 - 中国证监会周日晚间对其计划向商业银行发放券商牌照的报导做出回应。证监会表示，目前没有更多的信息需要向市场通报；而关于如何推进发展高质量投资银行，有多种路径选择，现尚在讨论中。 
证监会官方微信号并指出，发展高质量投资银行是贯彻落实国务院关于资本市场发展决策部署的需要，也是推进和扩大直接融资的重要手段。关于如何推进，有多种路

URL:https://www.reuters.com/article/usa-economy-may-consumer-income-0627-idCNKBS23Y014
TITLE:综述：美国5月消费者支出创史上最大涨幅 但个人收入下降令涨势难以为继
BODY:路透华盛顿6月26日 - 美国5月消费者支出录得有史以来最大反弹，但涨势不太可能持续，因收入下滑，且随着数以千万计的人将从下月开始失去失业补助，预计收入将进一步下滑。 
美国商务部周五报告的支出飙升，也受到美国许多地区新冠病毒确诊病例激增的威胁，这些地区包括人口稠密的加州、德克萨斯州和佛罗里达州。6月下半月，新冠感染病例不断增加，打击了消费者信心。6月，人们对政府经济政策的信心降至特朗普总统入主白宫以来的最低水平。 
美国经济正显现出好转的迹象，此前为控制新冠大流行而采取的严厉措施，导致经济在2月陷入衰退。5月就业岗位，房屋建筑许可、工业生产和制成品订单均反弹，部分收复了此前创下的历史性跌幅。      
“未来的经济仍有巨大的隐患，”PNC Financial的首席经济学家Gus Faucher表示，”个人收入和消费者支出在未来几个月可能会受到巨大打击，除非国会提供更多的财政刺激措施。” 
美国商务部说，占美国经济活动三分之二以上的消费者支出上个月增长了8.2%。这是自1959年政府开始跟踪该系列数据以来的最大增幅。4月消费者支出历史性地下跌了12.6%。 
接受路透调查的经济学家此前预计5月消费者支出将增长9.0%。上月支出激增反映出，在3月中旬为减缓疫情传播而实施封锁措施后，许多企业复工复产。 
消费者增加购买汽车和娱乐用品。他们还增加了医疗保健、餐馆、酒店和汽车旅馆的支出。 
但个人收入下降4.2%，为2013年1月以来最大降幅。今年4月，政府向数百万人发放了一次性1,200美元支票，并增加了失业补助，使个人收入飙升10.8%，创历史新高。这笔款项是一项近3万亿美元的史上最大规模财政刺激计划的一部分。 
在周五发布的另外一份报告中，密歇根大学表示，6月中旬消费者信心指数从78.9降至78.1。尽管消费者信心较5月上升，但冠状病毒病例创纪录增长地区的消费者信心不及东北地区，这可能令未来几个月的整体信心承压。 
受冠状病毒感染病例上升以及美国联邦储备理事会(FED/美联储)至少在第四季度之前限制银行大

URL:https://www.reuters.com/article/russia-constitutional-reform-vote-0629-idCNKBS24010G
TITLE:目前为止76%俄罗斯选民投票支持允许普京连任的修宪改革--出口民调
BODY:路透莫斯科6月29日 - 俄罗斯国营民调机构—全俄社会舆论中心(VTsIOM)周一表示，出口民调显示，目前为止76%俄罗斯民众投票支持允许总统普京连任至2036年的修宪改革。 
俄罗斯的修宪公投从6月25日开始投票，将持续七天。 
若相关改革措施获得通过，普京本次总统任期在2024年届满后，还可再连任两次。 
VTsIOM指出，目前为止已投票的民众之中，76%支持修宪改革，完成投票后同意接受民调的人之中，23.6%称他们投反对票。 
VTsIOM称已在俄罗斯25个地区的800个投票所，对163,124个选民进行民调。 
上述结果大致符合公投前根据VTsIOM先前民调所做的预测。(完) 
编译 蔡美珍；审校 孙茉莉
URL:https://www.reuters.com/article/china-mofa-hk-us-visa-restrictions-0629-idCNKBS2400VW
TITLE:中国外交部称中方决定对在涉港问题上表现恶劣的美方人员实施签证限制
BODY:路透北京6月29日 - 对于美国此前宣布对中国有关涉港官员实施签证限制，中国周一推出反制措施。外交部发言人赵立坚表示，中方决定，对在涉港问题上表现恶劣的美方人员实施签证限制。 
他在外交部例行发布会上称，香港国安立法属中国内政，任何外国无权干涉。美方通过所谓制裁阻挠中方推进香港国家安全立法的图谋不会得逞。 
“针对美方上述错误行径，中方决定，对在涉港问题上表现恶劣的美方人员实施签证限制。”他表示。 
当被追问中方签证限制措施涉及哪些人员时，赵立坚表示，具体指哪些人，”有关人员心知肚明”。 
赵立坚强调，中国政府维护国家主权、安全、发展利益的决心坚定不移,贯彻”一国两制”方针的决心坚定不移,反对任何外部势力干涉香港事务的决心坚定不移。 
美国国务卿蓬佩奥周五表示，美国将对据信对限制香港自由负有责任的中国官员实施签证限制，但未提及具体名单。 
此外，对于日前美国会参议院审议通过”香港自治法案”，赵立坚回应称，美国会参议院审议通过有关涉

URL:https://www.reuters.com/article/britain-sterling-idCNL4S2E62C5
TITLE:欧洲汇市：基建支出计划支撑英镑兑美元走平，但英退风险限制涨势
BODY:路透伦敦6月29日 - 英镑兑美元周一由上周五触及的一个月低点回升，英国首相约翰逊暗示，将加大力道推动提高公共投资的计划，但英国退欧风险令英镑涨势受限。 
约翰逊在电台专访中表示，在英国经济从新冠疫情的冲击中复苏之际，重拾财政紧缩将会是一场错误。 
英国内政大臣帕特尔(Priti Patel)周日表示，政府周二将公布一项提高基建支出的计划。 
“报导暗示，英镑受宣称的基础设施投资提振走强，”ING在给客户的报告中写道。报告并称，但对于英国能否和欧盟达成贸易协议的疑虑本周料是英镑最大的拖累。 
“我们看到英镑走势受到退欧谈判的左右，明天将开始新一轮谈判，”ING说。 
0840 GMT，英镑兑走软的美元持平于1.2340，此前美元指数从周五所及一周高点回落，因投资人担忧美国爆发新一轮疫情。 
英镑兑欧元下跌0.3%至0.9120英镑，接近周五所及的3月26日以来最低。（完） (编译 蔡美珍/李春喜 审校 郑茵/张明钧)


In [112]:
#繼承
class Webpage:
    def __init__(self, pageType, name, url, searchUrl, resultListing,
                resultUrl, absoluteUrl, titleTag, bodyTag):
        self.name = name
        self.url = url
        self.titleTag = titleTag
        self.bodyTag = bodyTag
        self.pageType = pageType
        
class Webpage:
    def __init__(self, name, url, titleTag):
        self.name = name
        self.url = url
        self.titleTag = titleTag
            
class Product(Website):
    def __init__(self, name, url, titleTag, productTagNumber, price):
        Website.__init__(name, url, titleTag)
        self.productTagNumber = productTagNumber
        self.price = price
        
class Article(Website):
    def __init__(self, name, url, titleTag, bodyTag, dateTag):
        Website.__init__(name, url, titleTag, bodyTag)
        self.dateTag = dateTag